# Testing mlbox on HousePrice Prediction

 Leonardo Ferreira 2018-07-09

## MLBox 
AutoML is a automated library and it supports distributed data processing, cleaning, formatting, and state-of-the-art algorithms such as LightGBM and XGBoost. It also supports model stacking, which allows you to combine an information ensemble of models to generate a new model aiming to have better performance than the individual models. <br> website: http://mlbox.readthedocs.io/en/latest/

In [1]:
## Importing the librarys

In [2]:
from mlbox.preprocessing import *
from mlbox.optimisation import *
from mlbox.prediction import *

Using Theano backend.
/home/leonardo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/leonardo/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
# Seting the variables to before reading

In [4]:
paths = ["train.csv", "test.csv"] #to modify
target_name = "SalePrice" #to modify

In [5]:
# Reading the datasets and setting as argument the paths and target
data = Reader(sep=",").train_test_split(paths, target_name)  #reading

#Drift_thresholder will help you to drop IDs and drifting variables between train and test datasets:
data = Drift_thresholder().fit_transform(data)  #deleting non-stable variables


reading csv : train.csv ...
cleaning data ...
CPU time: 0.5867280960083008 seconds

reading csv : test.csv ...
cleaning data ...
CPU time: 0.44849133491516113 seconds

> Number of common features : 80

gathering and crunching for train and test datasets ...
reindexing for train and test datasets ...
dropping training duplicates ...
dropping constant variables on training set ...

> Number of categorical features: 43
> Number of numerical features: 37
> Number of training samples : 1460
> Number of test samples : 1459

> Top sparse features (% missing values on train set):
PoolQC         99.5
MiscFeature    96.3
Alley          93.8
Fence          80.8
FireplaceQu    47.3
dtype: float64

> Task : regression
count      1460.000000
mean     180921.195890
std       79442.502883
min       34900.000000
25%      129975.000000
50%      163000.000000
75%      214000.000000
max      755000.000000
Name: SalePrice, dtype: float64

computing drifts ...
CPU time: 1.5090322494506836 seconds

> Top 10

In [6]:
## Optimiser will optimize the hyperparameters based on the scoring 
opt = Optimiser(scoring = "r2", n_folds = 5)

/home/leonardo/anaconda3/lib/python3.6/site-packages/mlbox/optimisation/optimiser.py:77: UserWarning: Optimiser will save all your fitted models into directory 'save/joblib'. Please clear it regularly.
  +str(self.to_path)+"/joblib'. Please clear it regularly.")


In [7]:

opt.evaluate(None, data)


No parameters set. Default configuration is tested

##################################################### testing hyper-parameters... #####################################################

>>> NA ENCODER :{'numerical_strategy': 'mean', 'categorical_strategy': '<NULL>'}

>>> CA ENCODER :{'strategy': 'label_encoding'}

>>> ESTIMATOR :{'strategy': 'LightGBM', 'boosting_type': 'gbdt', 'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_bin': 255, 'max_depth': -1, 'min_child_samples': 10, 'min_child_weight': 5, 'min_split_gain': 0, 'n_estimators': 500, 'nthread': -1, 'num_leaves': 31, 'objective': 'regression', 'reg_alpha': 0, 'reg_lambda': 0, 'seed': 0, 'silent': True, 'subsample': 0.9, 'subsample_for_bin': 50000, 'subsample_freq': 1}


MEAN SCORE : r2 = 0.5484946719476428
VARIANCE : 0.042245273840640335 (fold 1 = 0.5208558581790057, fold 2 = 0.589333919987618, fold 3 = 0.5603071364331653, fold 4 = 0.48103728657621403, fold 5 = 0.5909391585622109)
CPU time: 5.336984872817993 seconds



0.5484946719476428

######################<br>
LEGEND: <br>
ne = Numerical Encoders <br>
ce = Categorical Encoders<br>
fs = Fature Slection<br>
est = Estimator<br>

####################<br>
<br>
we also have search and space as parameters of our space as the syntax:
    <br>
search can be ~choice~ to sparse values given as a list or ~uniform~ if give as [0.1, 10], start to end values.


In [8]:
space ={
    'ne__numerical_strategy'    :{"search":"choice",
                                  "space":[0,'mean','median','most_frequent']},
    'ne__categorical_strategy'  :{"search":"choice",
                                  "space":[np.NaN,"None"]},
    'ce__strategy'              :{"search":"choice",
                                  "space":['label_encoding','entity_embedding','dummification']},
    'fs__strategy'              :{"search":"choice",
                                  "space":['l1','variance','rf_feature_importance']},
    'fs__threshold'             :{"search":"uniform",
                                  "space":[0.01, 0.02, 0.03, 0.05, 0.08, 0.1]},
    'est__strategy'             :{"search":"choice",
                                  "space":["XGBoost"]},
    'est__max_depth'            :{"search":"choice",
                                  "space":[2,3,4,5]},
    'est__learning_rate'        :{"search":"uniform",
                                  "space":[0.001, 0.01,0.1, 1, 10 ]}, 
    'est__subsample'            :{"search":"uniform",
                                  "space":[i/10.0 for i in range(6,10)]},
    'est__colsample_bytree'     :{"search":"uniform",
                                  "space":[i/10.0 for i in range(6,10)]},
    'est__gamma'                :{"search":"uniform",
                                  "space":[i/10.0 for i in range(0,5)]},
    'est__reg_alpha'            :{"search":"uniform",
                                  "space":[01e-5, 1e-2, 0.1, 1, 100]},
    'est__reg_lambda'           :{"search":"uniform",
                                  "space":[0.001,0.1,0,1,10]},
    'est__n_estimators'         :{"search":"choice",
                                  "space":[300, 500, 1000,1250,1500]}
}

In [9]:
best = opt.optimise(space, data, 120)


##################################################### testing hyper-parameters... #####################################################

>>> NA ENCODER :{'numerical_strategy': 0, 'categorical_strategy': nan}

>>> CA ENCODER :{'strategy': 'label_encoding'}

>>> FEATURE SELECTOR :{'strategy': 'l1', 'threshold': 0.030365057184518277}

>>> ESTIMATOR :{'strategy': 'XGBoost', 'colsample_bytree': 0.8449002356448905, 'learning_rate': 5.599322329230557, 'max_depth': 5, 'n_estimators': 1500, 'reg_alpha': 75.49793668219834, 'reg_lambda': 3.86066705419693, 'subsample': 0.8391882912488107, 'base_score': 0.5, 'colsample_bylevel': 1.0, 'gamma': 0, 'max_delta_step': 0, 'min_child_weight': 1, 'missing': None, 'nthread': -1, 'objective': 'reg:linear', 'scale_pos_weight': 1, 'seed': 0, 'silent': True}



/home/leonardo/anaconda3/lib/python3.6/site-packages/mlbox/model/supervised/regression/regressor.py:98: UserWarning: Invalid parameter for regressor LightGBM. Parameter IGNORED. Check the list of available parameters with `regressor.get_params().keys()`
  ". Parameter IGNORED. Check the list of "
/home/leonardo/anaconda3/lib/python3.6/site-packages/mlbox/optimisation/optimiser.py:428: UserWarning: An error occurred while computing the cross validation mean score. Check the parameter values and your scoring function.
  warnings.warn("An error occurred while computing the cross "
/home/leonardo/anaconda3/lib/python3.6/site-packages/mlbox/model/supervised/regression/regressor.py:98: UserWarning: Invalid parameter for regressor LightGBM. Parameter IGNORED. Check the list of available parameters with `regressor.get_params().keys()`
  ". Parameter IGNORED. Check the list of "



MEAN SCORE : r2 = -inf
VARIANCE : nan (fold 1 = -inf, fold 2 = -inf, fold 3 = -inf, fold 4 = -inf, fold 5 = -inf)
CPU time: 0.3878345489501953 seconds


##################################################### testing hyper-parameters... #####################################################

>>> NA ENCODER :{'numerical_strategy': 'median', 'categorical_strategy': 'None'}

>>> CA ENCODER :{'strategy': 'dummification'}

>>> FEATURE SELECTOR :{'strategy': 'l1', 'threshold': 0.0970113171133482}

>>> ESTIMATOR :{'strategy': 'XGBoost', 'colsample_bytree': 0.809196426190741, 'learning_rate': 7.072509605781133, 'max_depth': 5, 'n_estimators': 1000, 'reg_alpha': 82.20074372026168, 'reg_lambda': 7.967873283742195, 'subsample': 0.6058954843200101, 'base_score': 0.5, 'colsample_bylevel': 1.0, 'gamma': 0, 'max_delta_step': 0, 'min_child_weight': 1, 'missing': None, 'nthread': -1, 'objective': 'reg:linear', 'scale_pos_weight': 1, 'seed': 0, 'silent': True}



/home/leonardo/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/leonardo/anaconda3/lib/python3.6/site-packages/mlbox/optimisation/optimiser.py:428: UserWarning: An error occurred while computing the cross validation mean score. Check the parameter values and your scoring function.
  warnings.warn("An error occurred while computing the cross "
/home/leonardo/anaconda3/lib/python3.6/site-packages/mlbox/model/supervised/regression/regressor.py:98: UserWarning: Invalid parameter for regressor LightGBM. Parameter IGNORED. Check the list of available parameters with `regressor.get_params().keys()`
  ". Parameter IGNORED. Check the list of "



MEAN SCORE : r2 = -inf
VARIANCE : nan (fold 1 = -inf, fold 2 = -inf, fold 3 = -inf, fold 4 = -inf, fold 5 = -inf)
CPU time: 4.24631142616272 seconds


##################################################### testing hyper-parameters... #####################################################

>>> NA ENCODER :{'numerical_strategy': 'median', 'categorical_strategy': 'None'}

>>> CA ENCODER :{'strategy': 'label_encoding'}

>>> FEATURE SELECTOR :{'strategy': 'l1', 'threshold': 0.054585118061878}

>>> ESTIMATOR :{'strategy': 'XGBoost', 'colsample_bytree': 0.6456827694774263, 'learning_rate': 0.43842338961778443, 'max_depth': 3, 'n_estimators': 500, 'reg_alpha': 95.73223233549533, 'reg_lambda': 1.0592452424598109, 'subsample': 0.7856255853871873, 'base_score': 0.5, 'colsample_bylevel': 1.0, 'gamma': 0, 'max_delta_step': 0, 'min_child_weight': 1, 'missing': None, 'nthread': -1, 'objective': 'reg:linear', 'scale_pos_weight': 1, 'seed': 0, 'silent': True}



...

/home/leonardo/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/leonardo/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/leonardo/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/leonardo/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective di


MEAN SCORE : r2 = 0.9010286301437265
VARIANCE : 0.03811554741970726 (fold 1 = 0.9307961290917246, fold 2 = 0.9117256486052092, fold 3 = 0.8906607354056835, fold 4 = 0.8325806559414556, fold 5 = 0.9393799816745593)
CPU time: 44.28027820587158 seconds


##################################################### testing hyper-parameters... #####################################################

>>> NA ENCODER :{'numerical_strategy': 'mean', 'categorical_strategy': 'None'}

>>> CA ENCODER :{'strategy': 'dummification'}

>>> FEATURE SELECTOR :{'strategy': 'l1', 'threshold': 0.08581687364474215}

>>> ESTIMATOR :{'strategy': 'XGBoost', 'colsample_bytree': 0.789715914220416, 'learning_rate': 2.8182297204789237, 'max_depth': 5, 'n_estimators': 1250, 'reg_alpha': 20.506382194522686, 'reg_lambda': 2.878557254830746, 'subsample': 0.8545512907027247, 'base_score': 0.5, 'colsample_bylevel': 1.0, 'gamma': 0, 'max_delta_step': 0, 'min_child_weight': 1, 'missing': None, 'nthread': -1, 'objective': 'reg:line

/home/leonardo/anaconda3/lib/python3.6/site-packages/mlbox/model/supervised/regression/regressor.py:98: UserWarning: Invalid parameter for regressor LightGBM. Parameter IGNORED. Check the list of available parameters with `regressor.get_params().keys()`
  ". Parameter IGNORED. Check the list of "
/home/leonardo/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/leonardo/anaconda3/lib/python3.6/site-packages/mlbox/optimisation/optimiser.py:428: UserWarning: An error occurred while computing the cross validation mean score. Check the parameter values and your scoring function.
  warnings.warn("An error occurred while computing the cross "
/home/leonardo/anaconda3/lib/python3.6/site-packages/mlbox/model/supervised/regression/regressor.py:98: UserWarning: Invalid p


MEAN SCORE : r2 = -inf
VARIANCE : nan (fold 1 = -inf, fold 2 = -inf, fold 3 = -inf, fold 4 = -inf, fold 5 = -inf)
CPU time: 3.465012788772583 seconds


##################################################### testing hyper-parameters... #####################################################

>>> NA ENCODER :{'numerical_strategy': 'mean', 'categorical_strategy': nan}

>>> CA ENCODER :{'strategy': 'dummification'}

>>> FEATURE SELECTOR :{'strategy': 'l1', 'threshold': 0.07092838025323596}

>>> ESTIMATOR :{'strategy': 'XGBoost', 'colsample_bytree': 0.8506749580404083, 'learning_rate': 4.780791057915323, 'max_depth': 5, 'n_estimators': 1250, 'reg_alpha': 41.09917271262567, 'reg_lambda': 2.016277751405274, 'subsample': 0.8222197395802532, 'base_score': 0.5, 'colsample_bylevel': 1.0, 'gamma': 0, 'max_delta_step': 0, 'min_child_weight': 1, 'missing': None, 'nthread': -1, 'objective': 'reg:linear', 'scale_pos_weight': 1, 'seed': 0, 'silent': True}


MEAN SCORE : r2 = -inf
VARIANCE : nan (fold 1 =

/home/leonardo/anaconda3/lib/python3.6/site-packages/mlbox/optimisation/optimiser.py:428: UserWarning: An error occurred while computing the cross validation mean score. Check the parameter values and your scoring function.
  warnings.warn("An error occurred while computing the cross "


In [10]:
prd = Predictor()
prd.fit_predict(best, data)

/home/leonardo/anaconda3/lib/python3.6/site-packages/mlbox/model/supervised/regression/regressor.py:98: UserWarning: Invalid parameter for regressor LightGBM. Parameter IGNORED. Check the list of available parameters with `regressor.get_params().keys()`
  ". Parameter IGNORED. Check the list of "



fitting the pipeline ...


/home/leonardo/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


CPU time: 10.653747797012329 seconds

predicting...


/home/leonardo/anaconda3/lib/python3.6/site-packages/mlbox/prediction/predictor.py:386: UserWarning: Unable to get feature importances !
  warnings.warn("Unable to get feature importances !")


CPU time: 0.8523459434509277 seconds

> Overview on predictions : 

   SalePrice_predicted
0        124595.546875
1        157030.859375
2        186048.703125
3        197343.640625
4        183292.500000
5        173502.437500
6        172602.703125
7        167457.359375
8        181305.906250
9        125234.679688

dumping predictions into directory : save ...


In [13]:
df_test = pd.read_csv("test.csv" )
submission = pd.read_csv("SalePrice_predictions.csv")

In [14]:
my_submission = pd.DataFrame({'Id': df_test.Id, 'SalePrice': submission['SalePrice_predicted']})
# you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)